In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
data2 = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
data.head(10)

In [ ]:
data.shape

In [ ]:
data.count()

Il n'y a pas des données manquantes

In [ ]:
sns.pairplot(data)

In [ ]:
plt.figure(figsize=(12,12))
sns.kdeplot(data[data['Outcome'] == 0].Insulin, data[data['Outcome'] == 0].Glucose, cmap="Greens",  shade=True, alpha=0.3, shade_lowest=False)
sns.kdeplot(data[data['Outcome'] == 1].Insulin, data[data['Outcome'] == 1].Glucose, cmap="Reds",  shade=True, alpha=0.3, shade_lowest=False)

On observe ceux qui ont l'insulin et la glucose plus haute, tend plus à avoir diabète.

In [ ]:
data.columns

In [ ]:
sns.lmplot(x="Insulin", y="Glucose", data=data[data.Insulin != 0], fit_reg=False, hue='Outcome')

In [ ]:
sns.distplot(data.Insulin, color='blue')

In [ ]:
sns.kdeplot(data.Insulin, color='green')
sns.kdeplot(data.BloodPressure, color='red')
sns.kdeplot(data.SkinThickness, color='blue')

On voit qu'il y a une forte différente de distribution entre les trois séries. On va normalisé les distributions

In [ ]:
from sklearn import preprocessing

In [ ]:
minmax = preprocessing.MinMaxScaler(feature_range=(0, 1))
data[['SkinThickness', 'BloodPressure', 'Insulin']] = minmax.fit_transform(data[['SkinThickness', 'BloodPressure', 'Insulin']])

scaler = preprocessing.StandardScaler()
data[['SkinThickness', 'BloodPressure', 'Insulin']] = scaler.fit_transform(data[['SkinThickness', 'BloodPressure', 'Insulin']])

In [ ]:
sns.kdeplot(data.Insulin, color='green')
sns.kdeplot(data.BloodPressure, color='red')
sns.kdeplot(data.SkinThickness, color='blue')

# Machine Learning

In [ ]:
X = data.drop(['Outcome'], axis=1)
y = data.Outcome


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

In [ ]:
# Importation des méthodes de mesure de performances
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
print(confusion_matrix(y_test,y_lr))

In [ ]:
print(accuracy_score(y_test,y_lr))

In [ ]:
print(classification_report(y_test, y_lr))

In [ ]:
probas = lr.predict_proba(X_test)

dfprobas = pd.DataFrame(probas,columns=['proba_0','proba_1'])
dfprobas['y'] = np.array(y_test)

plt.figure(figsize=(10,10))
sns.distplot(1-dfprobas.proba_0[dfprobas.y==0], bins=50)
sns.distplot(dfprobas.proba_1[dfprobas.y==1], bins=50)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

In [ ]:
plt.figure(figsize=(12,12))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

* On obtien un AUC de 0.84 c'est un modele qui a un bon résultat.

## Random Forest

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

print(classification_report(y_test, y_rf))

In [ ]:
print(accuracy_score(y_test,y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
rf1 = ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=10, max_features=3)
rf1.fit(X_train, y_train)
y_rf1 = rf.predict(X_test)
print(classification_report(y_test, y_rf1))

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'min_samples_leaf', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('min_samples_leaf')
plt.ylabel('score');

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'max_features', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

In [ ]:
from sklearn import model_selection

param_grid = {
              'n_estimators': [10, 120, 200],
              'min_samples_leaf': [1, 30, 100]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

In [ ]:
rf_gs.fit(X_train, y_train)

In [ ]:
print(rf_gs.best_params_)

In [ ]:
rf2 = rf_gs.best_estimator_

y_rf2 = rf2.predict(X_test)
print(classification_report(y_test, y_rf2))

In [ ]:
print(accuracy_score(y_test,y_rf2))

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')

# XGB

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))

Les meilleurs résultat c'ést de 0.81 de accuracy f1-score avec random forest.